In [1]:
# S-Box (fixed mapping based on the table)
S_BOX = {
    0x0: 0xE, 0x1: 0x4, 0x2: 0xD, 0x3: 0x1,
    0x4: 0x2, 0x5: 0xF, 0x6: 0xB, 0x7: 0x8,
    0x8: 0x3, 0x9: 0xA, 0xA: 0x6, 0xB: 0xC,
    0xC: 0x5, 0xD: 0x9, 0xE: 0x0, 0xF: 0x7
}

#P-box permutation (transposition of bits)
P_BOX = [1, 5, 9, 13, 2, 6, 10, 14, 3, 7, 11, 15, 4, 8, 12, 16]

# Example Subkeys
SUBKEYS = [0x1F2A, 0x3C4D, 0x5E6F]

def s_box_substituion(nibble):
    """ Substitutes a 4-bit nibbke using the S-Box """
    return S_BOX[nibble]

def permutaion(block):
    """ Permutes a 16-bit block using the P-box """
    return sum(((block >> (i)) & 1) << (P_BOX[i] - 1) for i in range(16))

def key_mixing(block, subkey):
    """ Mixes a 16-bit block with a 16-bit subkey """
    return block ^ subkey

def round_function(block, subkey):
    """ Performs the operation of a single round: Substitution, Permutation, and Key Mixing """
    # Split block into 4 nibbles
    nibbles = [(block >> (i * 4)) & 0xF for i in range(4)]
    
    # Apply substitution to each nibble
    substituted = [s_box_substituion(nibble) for nibble in nibbles]
    
    # Concatenate the nibbles into a 16-bit block
    substituted_block = sum(substituted[i] << (i * 4) for i in range(4))
    
    # Permute the 16-bit block
    block = permutaion(substituted_block)
    
    # Mix the block with the subkey
    mix_block = key_mixing(block, subkey)
    
    return mix_block

def encrypt_block(plaintext):
    """ Encrypts a single block of plaintext using SPN cipher """    
    block = plaintext
    for subkey in SUBKEYS:
        block = round_function(block, subkey)
    return block

if __name__ == "__main__":
    plaintext = 0x1234
    ciphertext = encrypt_block(plaintext)
    print(f"Plaintext: {hex(plaintext)}")
    print(f"Ciphertext: {hex(ciphertext)}")



# Inverse S-box (inverse mapping of provided S-box)
INVERSE_S_BOX = {v: k for k, v in S_BOX.items()}

def inv_s_box_substitution(nibble):
    """Substitutes a 4-bit nibble using the inverse S-Box"""
    return INVERSE_S_BOX[nibble]

def inverse_permutation(block):
    """Permutes a 16-bit block using the inverse P-box"""
    return sum(((block >> (P_BOX.index(i))) & 1) << (i - 1) for i in range(1, 17))

def inv_round_function(block, subkey):
    """Performs the inverse operation of a single round"""
    # First, undo the key mixing (XOR is its own inverse)
    block = block ^ subkey
    
    # Undo the permutation
    block = inverse_permutation(block)
    
    # Split block into 4 nibbles
    nibbles = [(block >> (i * 4)) & 0xF for i in range(4)]
    
    # Apply inverse substitution to each nibble
    inverted = [inv_s_box_substitution(nibble) for nibble in nibbles]
    
    # Concatenate the nibbles into a 16-bit block
    block = sum(inverted[i] << (i * 4) for i in range(4))
    
    return block

def decrypt(ciphertext):
    """Decrypts a single block of ciphertext using SPN cipher"""
    block = ciphertext
    
    # Process in reverse order of subkeys
    for subkey in reversed(SUBKEYS):
        block = inv_round_function(block, subkey)
    
    return block

if __name__ == "__main__":
    plaintext = 0x1234
    ciphertext = encrypt_block(plaintext)
    decrypted_text = decrypt(ciphertext)
    print(f"Plaintext: {hex(plaintext)}")
    print(f"Ciphertext: {hex(ciphertext)}")
    print(f"Decrypted Text: {hex(decrypted_text)}")

Plaintext: 0x1234
Ciphertext: 0x8128
Plaintext: 0x1234
Ciphertext: 0x8128
Decrypted Text: 0x1234


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model, layers
import random
from tqdm import tqdm

class DifferentialDistinguisher:
    def __init__(self, input_size=32):  # 32 bits total (16 bits each for c and c')
        self.input_size = input_size
        self.model = self._build_model()
    
    def _build_model(self):
        inputs = layers.Input(shape=(self.input_size,))
        x = layers.Dense(64, activation='relu')(inputs)
        x = layers.Dense(32, activation='relu')(x)
        x = layers.Dense(16, activation='relu')(x)
        outputs = layers.Dense(1, activation='sigmoid')(x)
        
        model = Model(inputs=inputs, outputs=outputs)
        model.compile(optimizer='adam',
                     loss='binary_crossentropy',
                     metrics=['accuracy'])
        return model

def binary_to_bits(value, bits=16):
    """Convert an integer to its binary representation as a list of bits"""
    return [(value >> i) & 1 for i in range(bits)]

def bits_to_binary(bits):
    """Convert a list of bits back to an integer"""
    return sum(bit << i for i, bit in enumerate(bits))

def prepare_input(c1, c2):
    """Prepare the input for the neural network by converting to binary"""
    bits1 = binary_to_bits(c1)
    bits2 = binary_to_bits(c2)
    return np.array(bits1 + bits2)

class CustomCallback(tf.keras.callbacks.Callback):
    def __init__(self, epochs):
        super(CustomCallback, self).__init__()
        self.epochs_pbar = tqdm(total=epochs, desc='Training epochs', leave=True)

    def on_epoch_end(self, epoch, logs=None):
        self.epochs_pbar.update(1)
        self.epochs_pbar.set_postfix(loss=f"{logs['loss']:.4f}", acc=f"{logs['accuracy']:.4f}")

    def on_train_end(self, logs=None):
        self.epochs_pbar.close()

def train_network(n, epochs, delta_x):
    """Implementation of the Neural Network Training algorithm"""
    distinguisher = DifferentialDistinguisher()
    training_input = []
    training_output = []
    
    print("Generating training data...")
    for _ in tqdm(range(n), desc='Generating pairs'):
        if random.random() > 0.5:
            # Generate real differential pairs
            m = random.getrandbits(16)
            k = random.getrandbits(16)
            c = encrypt_block(m)
            # Apply delta_x to message
            c_prime = encrypt_block(m ^ delta_x)
            
            training_input.append(prepare_input(c, c_prime))
            training_output.append(1)
        else:
            # Generate random pairs
            c = random.getrandbits(16)
            c_prime = random.getrandbits(16)
            
            training_input.append(prepare_input(c, c_prime))
            training_output.append(0)
    
    training_input = np.array(training_input)
    training_output = np.array(training_output)
    
    print("\nTraining neural network...")
    distinguisher.model.fit(
        training_input,
        training_output,
        epochs=epochs,
        batch_size=32,
        verbose=0,
        callbacks=[CustomCallback(epochs)]
    )
    
    return distinguisher

def neural_distinguisher(delta_x, n, tau, epochs):
    """Implementation of the Neural distinguisher A3"""
    h = 0
    net = train_network(n, epochs, delta_x)
    
    print("\nTesting distinguisher...")
    for _ in tqdm(range(n), desc='Testing pairs'):
        m = random.getrandbits(16)
        c = encrypt_block(m)
        c_prime = encrypt_block(m ^ delta_x)
        
        input_data = prepare_input(c, c_prime)
        prediction = net.model.predict(np.array([input_data]), verbose=0)[0]
        
        if prediction >= 0.5:  # Threshold for binary classification
            h += 1
    
    return 1 if h >= tau else 0

def test_distinguisher():
    """Test function to demonstrate the usage"""
    # Parameters
    n = 1000          # Number of training samples
    epochs = 10       # Training epochs
    delta_x = 0x0040  # Example input difference
    tau = n // 2      # Threshold (adjust as needed)
    
    print("Starting Neural Distinguisher Test")
    print("-" * 40)
    print(f"Parameters:")
    print(f"n: {n}")
    print(f"epochs: {epochs}")
    print(f"delta_x: {hex(delta_x)}")
    print(f"tau: {tau}")
    print("-" * 40)
    
    # Run the distinguisher
    result = neural_distinguisher(delta_x, n, tau, epochs)
    
    print("\nResults:")
    print("-" * 40)
    print(f"Neural Distinguisher Output: {result}")
    print("-" * 40)

if __name__ == "__main__":
    test_distinguisher()